In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    resized_img = cv2.resize(img, (100, 100))
    return resized_img.flatten()

def load_data(genuine_folder, forged_folder):
    signatures = []
    labels = []

    # Load genuine signatures
    genuine_images = [img for img in os.listdir(genuine_folder)]
    for genuine_image in genuine_images:
        genuine_path = os.path.join(genuine_folder, genuine_image)
        signatures.append(preprocess_image(genuine_path))
        labels.append('genuine')

    # Load forged signatures
    forged_images = [img for img in os.listdir(forged_folder)]
    for forged_image in forged_images:
        forged_path = os.path.join(forged_folder, forged_image)
        signatures.append(preprocess_image(forged_path))
        labels.append('forged')

    return np.array(signatures), np.array(labels)


# signature folders
genuine_folder = r"C:\Users\ahmed\OneDrive\Documents\Artificial Intelligence\Genuine_1"
forged_folder = r"C:\Users\ahmed\OneDrive\Documents\Artificial Intelligence\Forged_1"

# Load data
signatures, labels = load_data(genuine_folder, forged_folder)

# Display information about the data being used for training
num_genuine = len([label for label in labels if label == 'genuine'])
num_forged = len([label for label in labels if label == 'forged'])

print(f"Number of genuine signatures: {num_genuine}")
print(f"Number of forged signatures: {num_forged}")
print(f"Number of data points used for training: {len(signatures)}")

# Check if the data is sufficient for training
if num_genuine < 2 or num_forged < 2:
    print("Insufficient data for training. Please ensure there are at least two genuine and two forged signatures.")
    exit()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(signatures, labels, test_size=0.2, random_state=42)

# Check if the data is sufficient for testing
if len(set(y_train)) < 2 or len(set(y_test)) < 2:
    print("Insufficient data for testing. Please ensure there are both genuine and forged signatures in the training and test sets.")
    exit()

# Creating and training a Support Vector Machine (SVM) model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on test set: {accuracy}")

# Input a test signature for verification whether if it's genuine or forged
input_signature_path = r"C:\Users\ahmed\OneDrive\Documents\Artificial Intelligence\test_signature.png"
verification_signature = preprocess_image(input_signature_path)
verification_result = svm_model.predict([verification_signature])[0]

# Displaying verification result
if verification_result == 'genuine':
    print("The signature is genuine.")
else:
    print("The signature is forged.")
